In [ ]:
#import libraries
import keras
import os
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import tensorflow as tf
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, Dataset

In [ ]:
from google.colab import drive # to mount google drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
ls /content/drive/My\ Drive/

In [ ]:
!unzip /content/drive/My\ Drive/Bird_detection.zip

In [ ]:
DIR_TRAIN = "train"
DIR_VALID = "valid"
DIR_TEST = "test"

#print('number of male training images - ',len(os.listdir(DIR_TRAIN)))

In [ ]:
#classes =  len(os.listdir(DIR_TRAIN))
#print('number of male training images - ',classes)
### Exploring Dataset
train_imgs = []
valid_imgs = []
test_imgs = []
classes = os.listdir(DIR_TRAIN)
print("Total Classes: ",len(classes))

#Counting total train, valid & test images

train_count = 0
valid_count = 0
test_count = 0
for _class in classes:
  
    train_count += len(os.listdir(DIR_TRAIN +"/"+ _class))
    valid_count += len(os.listdir(DIR_VALID +"/"+ _class))
    test_count += len(os.listdir(DIR_TEST +"/"+ _class))

print("Total train images: ",train_count)
print("Total valid images: ",valid_count)
print("Total test images: ",test_count)

In [ ]:
train = os.listdir('train')
for _class in train:
    
    for img in os.listdir(DIR_TRAIN + "/" + _class):
        train_imgs.append(DIR_TRAIN + "/" + _class + "/" + img)
    
    for img in os.listdir(DIR_VALID + "/" + _class):
        valid_imgs.append(DIR_VALID  + "/" + _class + "/" + img)
        
    for img in os.listdir(DIR_TEST + "/" + _class):
        test_imgs.append(DIR_TEST  + "/" + _class + "/" + img)
class_to_int = {classes[i] : i for i in range(len(classes))}

In [ ]:
### Loading Classification Dataset - FOR METHOD 2: For multi-class data, by inheriting Dataset class


def get_transform():
    return T.Compose([T.ToTensor()])

class BirdDataset(Dataset):
    
    def __init__(self, imgs_list, class_to_int, transforms = None):
        
        super().__init__()
        self.imgs_list = imgs_list
        self.class_to_int = class_to_int
        self.transforms = transforms
        
        
    def __getitem__(self, index):
    
        image_path = self.imgs_list[index]
        
        #Reading image
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = cv2.resize(image, (224, 224))
        height, width, channels = image.shape

        
        
        #Retriving class label
        label = image_path.split("/")[-2]
        label = self.class_to_int[label]
        
        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
        
        return image

In [ ]:
train_dataset = BirdDataset(train_imgs, class_to_int, get_transform())
valid_dataset = BirdDataset(valid_imgs, class_to_int, get_transform())
test_dataset = BirdDataset(test_imgs, class_to_int, get_transform())

In [ ]:
#Creating generator for Training DataSet
#Reading image
        #image_path='train/'
        #image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        #image /= 255.0
         #image = cv2.resize(image, (224, 224))
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        './train/',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical')

#Creating generator for Validation DataSet
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_datagen.flow_from_directory(
        './valid/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

#Creating generator for Test DataSet
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

In [ ]:
#instantiate a base model with pre-trained weigts.
base_model=keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3))

In [ ]:
#freeze the base model
base_model.trainable = False

In [ ]:
#Create new model on top
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(225,activation='softmax',kernel_initializer='glorot_normal'))

In [ ]:
model.summary()

In [ ]:
#Train the model on new data.
model.compile(optimizer=keras.optimizers.Adam(1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,epochs=30,validation_data=val_generator,workers=10)

In [ ]:
model.predict("gbjh.jpg")